### 信息检索导论大作业
#### ———— LlamaIndex框架实现增强检索的足球私域知识对话模型

2024-05-23

#### 一、准备工作

（1）OPENAI_API_KEY

In [1]:
import os

OPENAI_API_KEY = '' 
os.environ['OPENAI_API_KEY'] = '' 

from dotenv import load_dotenv

load_dotenv()

False

（2）读取documents，把文件用 SimpleDirectoryReader 方法存成 documents

In [2]:
# encoding:utf-8
import os
import gradio as gr
import openai
import pinecone

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.core.schema import IndexNode
# from llama_index.llama_predictor import llama_predictor

# from langchain import OpenAI
from llama_index.core import (
    GPTKeywordTableIndex,
    SimpleDirectoryReader,
    # LLMPredictor,
    ServiceContext
)

documents = SimpleDirectoryReader(input_dir='./data').load_data()

os.environ['OPENAI_API_KEY'] = '' 

ModuleNotFoundError: No module named 'gradio'

（3）建立三种嵌入方式的索引

index_OpenAIEmbeddings：使用的 embedding = OpenAIEmbeddings()参数

index_bge_large_zh：使用的 model_name="BAAI/bge-large-zh-v1.5"

index_bge_M3：使用的 model_name="BAAI/bge-M3"

In [4]:
from llama_index.core import VectorStoreIndex,DocumentSummaryIndex
# from haystack.indexing.vector_store import VectorStoreIndex
from llama_index.core import KnowledgeGraphIndex
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings()  
print("OpenAIEmbeddings:")
index_OpenAIEmbeddings = VectorStoreIndex.from_documents(documents = documents, embedding = OpenAIEmbeddings(), show_progress = 1)

from langchain.embeddings import HuggingFaceBgeEmbeddings

# # bge-large-zh-v1.5
# print("bge-large-zh-v1.5:")
# bge_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-zh-v1.5")
# index_bge_large_zh = VectorStoreIndex.from_documents(documents = documents, embedding = bge_embeddings, show_progress = 1)

# # bge-M3
# print("bge-M3:")
# bgeM3_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-M3")
# index_bge_M3 = VectorStoreIndex.from_documents(documents = documents, embedding = bgeM3_embeddings, show_progress = 1)


OpenAIEmbeddings:


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1277 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### 二、形成 engine, 对query进行回答

In [4]:
# query_engine = index_OpenAIEmbeddings.as_chat_engine(verbose=True)

# response_vector = query_engine.query("你是个啥模型？")

# response_vector

（1）embedding 使用 index_bge_M3, 无查询重写

In [5]:
# from llama_index.core.node_parser import SentenceSplitter

# node_parser = SentenceSplitter.from_defaults(chunk_size=512)
# nodes = node_parser.get_nodes_from_documents(
#     documents,show_progress=True)
# nodes

query_engine = index_bge_M3.as_chat_engine(verbose=True)

response_bge = query_engine.query("你是个啥模型？")

response_bge

Added user message to memory: 你是个啥模型？
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"我是一个自然语言处理模型，可以回答您关于各种主题的问题和提供信息。请告诉我您想了解的内容。"}
Got output: 您可以问我关于足球新闻、球队表现、球员表现、转会窗口等与足球相关的问题。



Response(response='我是一个自然语言处理模型，可以回答您关于足球新闻、球队表现、球员表现、转会窗口等与足球相关的问题。请告诉我您想了解的内容。', source_nodes=[NodeWithScore(node=TextNode(id_='b3ba4c76-33b0-42a4-9921-0b9b629e30c8', embedding=None, metadata={'file_path': 'c:\\Users\\Thinkpad\\Desktop\\项目大作业——基于 LLM 和 RAG 的足球新闻检索系统\\data\\data.csv', 'file_name': 'data.csv', 'file_type': 'application/vnd.ms-excel', 'file_size': 3953400, 'creation_date': '2024-04-12', 'last_modified_date': '2024-05-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='62b2c02b-e72f-4a5d-b80a-ac2dbf61f249', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'c:\\Users\\Thinkpad\\Desktop\\项目大作业——基于 LLM 和 RAG 的足球新闻检索系统\\data\\data.csv', 'file_name': 'data.csv', 'file_type': 'applica

（2）embedding使用index_bge_M3，采用Hyde查询重写

In [6]:
# Hyde查询重写的结果

from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

query_engine = index_bge_M3.as_chat_engine(verbose=True)

# Query without transformation: The same query string is used for embedding lookup and also summarization.
response = query_engine.query("曼联队员在3-0战胜西汉姆的比赛中都是怎样表现的？")

print('-' * 100)
print("Base query:")
print(response)
# index_bge_M3
# hyde 查询重写
# Query with HyDE transformation
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query("曼联队员在3-0战胜西汉姆的比赛中都是怎样表现的？")


print('-' * 100)
print("After HyDEQueryTransform:")
print(response)

Added user message to memory: 曼联队员在3-0战胜西汉姆的比赛中都是怎样表现的？
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"How did Manchester United players perform in the 3-0 victory against West Ham?"}
Got output: Manchester United players performed well in the 3-0 victory against West Ham. Rashford and Højlund scored early goals, with McTominay adding a third goal off the bench in the second half. Additionally, Kobbie Mainoo scored a late winner in stoppage time to secure the victory for Manchester United.

----------------------------------------------------------------------------------------------------
Base query:
曼联队员在3-0战胜西汉姆的比赛中表现出色。拉什福德和霍伦德早早进球，麦克托米奈在下半场替补出场时攻入第三球。此外，科比·迈努在补时阶段打入一球，为曼联取得了胜利。
Added user message to memory: 曼联队员在3-0战胜西汉姆的比赛中都是怎样表现的？
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"How did Manchester United players perform in the 3-0 victory over West Ham?"}
Got output: Manchester United players performed well in t

In [16]:
# 假设 documents 是一个包含文档对象的列表
# 定义空的文本列表和元数据列表
doc_texts = []

# 遍历每个文档对象，获取文本内容和元数据
for i, doc in enumerate(documents):
    text = doc.text  # 假设文档对象有一个 text 属性用于获取文本内容
    if text:
        doc_texts.append(text)
    else:
        doc_texts.append("")  # 如果文本内容为空，添加一个空字符串


splitted_texts = []
# 遍历每个文本内容，按照 "http" 进行分割
for text in doc_texts:
    # 使用 split 方法按照 "http" 进行分割，并加入到分割后的文本列表中
    splitted_texts.extend(text.split("http"))

print(len(splitted_texts))
# 打印分割后的文本列表
print(splitted_texts)

1725
['', 's://www.livemint.com/sports/football-news/lionel-messi-signs-fan-s-argentina-jersey-with-a-marker-pen-in-traffic-video-goes-viral-watch-11706370425816.html, Saurav Mukherjee, Lionel Messi signs fan’s Argentina jersey with a marker pen in traffic, video goes viral. Watch, The football superstar could hardly be seen saying no to his fans. In the latest, a video clip of Messi is going viral on social media in which he can be seen signing a fan\'s jersey in the middle of traffic. In the clip, shared on X, Messi can be seen signing a fan’s Argentina jersey with a marker pen, and that too while stuck in traffic, after it was tossed by a fan through Messi\'s passenger side window following pulling up next to the Inter Miami star while driving. ALSO READ: Argentina won’t allow anyone to wear Lionel Messi’s No. 10 jersey: ‘Least we can do for him’ After signing the jersey, Messi leaned over and handed it back to the fan through the same car window. The Argentinian footballer even sho

（3）普通embedding, 用 BM25 和 FAISS 混合检索

In [26]:

# 尝试混合检索方式
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 将文档转换为文本列表和元数据列表
# doc_texts = [doc.text for doc in documents]
doc_metadatas = [{"source": i} for i in range(len(splitted_texts))]
print(len(splitted_texts))

# 初始化BM25检索器
bm25_retriever = BM25Retriever.from_texts(splitted_texts, metadatas = doc_metadatas)
bm25_retriever.k = 3

# 初始化FAISS检索器
embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(splitted_texts, embedding, metadatas=doc_metadatas)

# 将FAISS向量存储转化为检索器
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 3})

# 初始化Ensemble Retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

# 使用Ensemble Retriever进行检索
query = "曼联队员在3-0战胜西汉姆的比赛中都是怎样表现的？"
docs = ensemble_retriever.invoke(query)
print(len(docs))

type(docs)

for doc in docs:
    print(f"Document ID: {doc.lc_id}")
    print(f"Content: {doc.page_content}")
    print("\n---\n")

1725
6
Document ID: <bound method Serializable.lc_id of <class 'langchain_core.documents.base.Document'>>
Content: s://theanalyst.com/eu/2024/01/west-ham-vs-brighton-prediction/, Tom Patey, West Ham vs Brighton: Prediction and Preview, Both in contention for European qualification once more, we look ahead to Tuesday night’s Premier League clash at London Stadium with our West Ham vs Brighton prediction and preview. West Ham United will welcome fellow European contenders Brighton and Hove Albion to celebrate the new year as the pair meet on Tuesday at London Stadium after two equally impressive Premier League results on Matchday 19. David Moyes managed his first ever away win against Arsenal in all competitions at the 23rd attempt on Thursday as West Ham recorded an unlikely 2-0 victory over Mikel Arteta’s title contenders. Tomás Soucek scored a contentious 13th-minute opener, with referee Michael Oliver and the VAR allowing the goal to stand despite claims the ball had gone out from Ja

Added user message to memory: 切尔西在2024年足总杯第三轮中以4-0战胜了哪支球队？请用中文回答

=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"切尔西在2024年足总杯第三轮以4-0战胜了哪支球队？"}
Got output: 切尔西在2024年足总杯第三轮以4-0战胜了谢菲联队。

Added user message to memory: 切尔西在2024年足总杯第三轮中以4-0战胜了哪支球队？

=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"Chelsea 4-0 which team in the FA Cup third round in 2024"}
Got output: Chelsea defeated Preston North End 4-0 in the FA Cup third round in 2024.



In [24]:
# 将相关文档内容组合成一个字符串，传递给query_engine
doc_contents = "\n".join([doc.page_content for doc in docs])

combined_query = f"我的问题是：{query}。我已知下面这些信息：{doc_contents}。请你根据这些内容回答。"

# 使用query_engine进行总结和回答
response = query_engine.query(combined_query)

# 打印结果
print(response)

Added user message to memory: 我的问题是：曼联队员在3-0战胜西汉姆的比赛中都是怎样表现的？。我已知下面这些信息：s://theanalyst.com/eu/2024/01/west-ham-vs-brighton-prediction/, Tom Patey, West Ham vs Brighton: Prediction and Preview, Both in contention for European qualification once more, we look ahead to Tuesday night’s Premier League clash at London Stadium with our West Ham vs Brighton prediction and preview. West Ham United will welcome fellow European contenders Brighton and Hove Albion to celebrate the new year as the pair meet on Tuesday at London Stadium after two equally impressive Premier League results on Matchday 19. David Moyes managed his first ever away win against Arsenal in all competitions at the 23rd attempt on Thursday as West Ham recorded an unlikely 2-0 victory over Mikel Arteta’s title contenders. Tomás Soucek scored a contentious 13th-minute opener, with referee Michael Oliver and the VAR allowing the goal to stand despite claims the ball had gone out from Jarrod Bowen’s assist – a similar decision to

### 三、利用Gradio创建引擎交互界面

【目前版本embedding采用index_OpenAIEmbeddings，未采用查询重写】

In [25]:
import gradio as gr
from llama_index.core import VectorStoreIndex
from langchain_openai import OpenAIEmbeddings
from gradio.components import Textbox


# Create a chat engine with verbose output
# 这个位置可以替换不同的嵌入模型搞的引擎
query_engine = index_OpenAIEmbeddings.as_chat_engine(verbose=True)

# Define the query function
def get_response(query):
    response = query_engine.query(query)
    return response


# 创建Gradio界面
iface = gr.Interface(
    fn=get_response,
    inputs=gr.components.Textbox(lines=5, label="输入提示文本"),
    outputs="text",
    title="小鹿的足球信息检索系统",
    description="使用 gpt-3.5 + 最新足球新闻的rag检索系统"
)

# 启动Gradio界面
iface.launch()



IMPORTANT: You are using gradio version 3.50.0, however version 4.29.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Added user message to memory: Petr Cech预测切尔西何时可能夺得英超联赛冠军
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"When did Petr Cech predict Chelsea could win the Premier League title?"}
Got output: Petr Cech predicted that Chelsea could win the Premier League title in the 2023-24 season.

Added user message to memory: 为什么不是2025-2026
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"Why is it not 2025-2026?"}
Got output: It is not 2025-2026 because Petr Cech mentioned that the earliest season Chelsea could win the league is the 2025-2026 season.

Added user message to memory: 在这次转会窗口，巴塞罗那计划如何处理财政问题以引进Amadou Onana？
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "What financial strategies is Barcelona planning to implement in order to sign Amadou Onana during this transfer window?"}
Got output: Barcelona is considering bargain-hunting for budget transfer options due to financial constraints, as se